# Deep Dive into LangChain

## Python-dotenv
è un modulo che permette di specificare variabili di ambiente in un file .env all'interno della directory del progetto.
Creo una chiave da open ai e la aggiungo a .env

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

# os.environ.get("OPEN_AI_KEY")

True

## Chat Models: GPT-3.5 Turbo and GPT-4

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

output = llm.invoke('Explaing quantum mechanics in one sentence.', model="gpt-3.5-turbo")
print(output.content)

Quantum mechanics is a branch of physics that studies the behavior of particles at the smallest scales, where traditional laws of physics break down and probabilities replace definitive outcomes.


In [3]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(BaseChatOpenAI)
 |  ChatOpenAI(
 |      *args: Any,
 |      name: Optional[str] = None,
 |      cache: Union[langchain_core.caches.BaseCache, bool, NoneType] = None,
 |      verbose: bool = <factory>,
 |      callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
 |      tags: Optional[list[str]] = None,
 |      metadata: Optional[dict[str, Any]] = None,
 |      custom_get_token_ids: Optional[Callable[[str], list[int]]] = None,
 |      callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
 |      rate_limiter: Optional[langchain_core.rate_limiters.BaseRateLimiter] = None,
 |      disable_streaming: Union[bool, Literal['tool_calling']] = False,
 |      client: Any = None,
 |      async_client: Any = None,
 |      root_client: Any = None,
 |      root_async_client: Any = None,
 |    

Quelli che più ci interessano sono:
`model: str = 'gpt-3.5-turbo',`
`temperature: Optional[float] = None,`

Quando usiamo le API di OpenAI (ad esempio con gpt-4 o gpt-3.5-turbo), il modello lavora con una lista di dizionari chiamata **messages**. Ogni dizionario rappresenta un messaggio con un ruolo specifico e il suo contenuto.

```
messages = [
    {"role": "system", "content": "Sei un assistente esperto di Python."}, # Definisce il comportamento e la personalità del modello.
    {"role": "user", "content": "Come faccio a leggere un file in Python?"}, # Contiene la richiesta dell’utente
    {"role": "assistant", "content": "Puoi usare la funzione `open()`. Ecco un esempio..."} # Contiene la risposta generata dall’IA.
]
```

Per interprearli dobbiamo richiamare tre classi da `langchain.schema`


In [4]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a physicist and respond only in German."),
    HumanMessage(content="Explaing quantum mechanics in one sentece.")
]

output = llm.invoke(messages)

print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene, indem sie sowohl Teilchen als auch Welleneigenschaften berücksichtigt.


## Caching LLM Responses
Nel contesto degli LLM, il caching ottimizza le interazioni riducendo le chiamate API e velocizzando l'applicazione, risultato in una esperienza utente più efficiente.

Abbiamo due tipi di caching:
- in memory cache
- sql lite cache

### In-Memory Cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [6]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler an understand'
llm.invoke(prompt)

CPU times: user 137 ms, sys: 58.3 ms, total: 195 ms
Wall time: 3.26 s


'\n\nWhy did the cookie go to the doctor?\nBecause it was feeling crumbly!'

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 505 μs, sys: 103 μs, total: 608 μs
Wall time: 727 μs


'\n\nWhy did the cookie go to the doctor?\nBecause it was feeling crumbly!'

### SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


# First requesst (not in cache, takes longer)
llm.invoke('Tell me a joke')

# Second Request (cached, faster)
llm.invoke('Tell me a joke')

## LLM Streaming
Nel contesto degli LLM, è il processo di consegnare una risposta in un continuo stream di dati invece di mandare l'intera risposta in una volta sola.

Questo permette all'utente di ricevere la risposta un pezzo alla volta mentre viene generata, questo può migliorare la user experience e ridurre la latenza complessiva.

In [8]:
# Senza streaming butterà tutto fuori in una volta
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the glowing moon
A raven flies high, singing a haunting tune
Its wings brush against the sky so dark
A creature of the night leaving its mark

Chorus:
The moon and the raven, together they soar
In the silence of the night, forevermore
Their spirits dance in the sky above
A rock and roll ode to the moon and the raven's love

Verse 2:
The moonlight shines down on the world below
As the raven watches from its blackened bough
Mysteries unfolding in the pale moon's glow
A dance of shadows in the night's soft flow

Chorus:
The moon and the raven, together they soar
In the silence of the night, forevermore
Their spirits dance in the sky above
A rock and roll ode to the moon and the raven's love

Bridge:
In the darkness, they find their solace
In the stillness of the night, their spirits race
A bond unbroken by time or space
The moon and the raven, a mysterious embrace

Chorus:
The moon and the raven, together they soar
In the silence of the night, foreverm

In [9]:
# Implementando lo streaming butterà un poco alla volta(nel senso che dividerà in chunk,
# per questo faremo un for loop
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

(Verse 1)
Underneath the silver light
The Moon shines bright in the dead of night
A raven flies across the sky
With a haunting cry, he catches my eye

(Chorus)
Moon and Raven, dancing in the dark
Mysteries of the night, leave their mark
Whispers of secrets untold
In the shadows, our stories unfold

(Verse 2)
The Moon guides the raven's flight
As they soar through the velvet night
In the silence of the unknown
They find solace in their own

(Chorus)
Moon and Raven, dancing in the dark
Mysteries of the night, leave their mark
Whispers of secrets untold
In the shadows, our stories unfold

(Bridge)
In the stillness of the night
They find comfort in each other's sight
A bond unbroken, a connection deep
In the darkness, their secrets keep

(Chorus)
Moon and Raven, dancing in the dark
Mysteries of the night, leave their mark
Whispers of secrets untold
In the shadows, our stories unfold

(Outro)
Moon and Raven, forever intertwined
In the night they'll always find
The peace and solace they seek

## Prompt Templates
Un prompt si riferisce a un input dato al modello.
**Prompt Templates** sono un modo di creare prompt dinamici per gli LLMs.

Un modello di prompt prende un pezzo di testo e inietta l'input di un utente in quel pezzo di testo.

In langchain esistono sia i **PromptTemplates** che i **ChatPromptTemplates**

In [10]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
# Simile alle f-string e usa le {}
template = '''You are an experience virologist.
Write a few sentences about the following virus {virus} in {language}.'''

# Questo creerà un obj dalla stringa che passiamo
prompt_template = PromptTemplate.from_template(template=template)

## Riformattiamo per sostituire i placeholder
prompt = prompt_template.format(virus='hiv', language='german')

prompt

'You are an experience virologist.\nWrite a few sentences about the following virus hiv in german.'

In [11]:
# Per ottenere la risposta
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt und zu AIDS führen kann. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es gibt keine Heilung für HIV, aber mit antiretroviralen Medikamenten kann die Krankheit kontrolliert werden. Prävention und frühzeitige Diagnose sind entscheidend im Kampf gegen HIV.


## Chat Prompt Templates
importiamo un pò di classi da langchain

In [22]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI 
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the json format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='Europe')
print(messages)

[SystemMessage(content='You respond only in the json format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in Europe by population.', additional_kwargs={}, response_metadata={})]


In [25]:
# Per ottenere la risposta

llm = ChatOpenAI()
# La chiamata corretta per questa versione
response = llm.invoke(messages)
print("\nRisposta:")
print(response.content)


Risposta:
{
    "1": {"country": "Germany", "population": 83149300},
    "2": {"country": "France", "population": 67146000},
    "3": {"country": "United Kingdom", "population": 65110000},
    "4": {"country": "Italy", "population": 60589445},
    "5": {"country": "Spain", "population": 47100396},
    "6": {"country": "Ukraine", "population": 44385155},
    "7": {"country": "Poland", "population": 38433600},
    "8": {"country": "Romania", "population": 23839700},
    "9": {"country": "Netherlands", "population": 17424978},
    "10": {"country": "Belgium", "population": 11589623}
}


## Simple Chains
Chains sono una serie di step e azioni.
Come una sequenza di chiamate dove ogni step può contenere implicare parlare con un modello, o preparare dati.

Ci permettono di combinare multipli componenti insieme per risolvere una task specifica e costruire un'intera applicazione LLM

In [30]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = ''''You are an experience virologist.
Write a few sentence about the following virus "{virus}" in {language}.
'''

prompt_template = PromptTemplate.from_template(template=template)

chain = prompt_template | llm

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})

In [31]:
print(output.content)

El virus del herpes simple (HSV) es un virus altamente contagioso que puede infectar a los humanos y causar una variedad de enfermedades, como el herpes labial y el herpes genital. Se transmite principalmente a través del contacto directo con lesiones activas en la piel u otras membranas mucosas. Aunque no se puede curar, existen tratamientos disponibles para controlar los síntomas y prevenir la transmisión a otras personas.


per rendere la risposta verbosa possiamo aggiungere al costruttore del chain il parametro `verbose=True`

Nell'esempio abbiamo due argomenti quindi passiamo un dizionario, ma con un solo argomento potremmo passare una variabile stringa.

In [36]:
template = 'What is the capital of {country}?, List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

chain = (prompt_template | llm).with_config({"verbose": True})
country = input('Enter Country')
output = chain.invoke({"country": country})

print(output.content)

Enter Country Italy


The capital of Italy is Rome.

- Colosseum
- Vatican City
- Pantheon


## Sequential Chains
Con le sequential chains posso fare una sequenza di chiamate a uno o più LLM. Posso prendere l'output da una chain e usarlo come input di un'altra chain.

Ci sono due tipi di **sequential chain**:
1. SimpleSequentialChain
2. General form of sequential chains

`SimpleSequentialChain` rapprensentano una sequenza di chain, dove ogni chain individuale ha un singolo input e un singolo output, e l'output di uno step viene usato come input del successivo

In [38]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature=0.5) # una temperatura più bassa indica una risposta più precisa
prompt_template1 = PromptTemplate.from_template(
    template='You are and experienced scientis and Python programmer. Writer a function that implements the concet of {concept}'
)
chain1 = LLMChain(llm = llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model_name = 'gpt-4-turbo-preview', temperature=1.2) # una temperatura più bassa indica una risposta più creativa
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)

chain2 = LLMChain(llm = llm2, prompt=prompt_template2)

# Ora creaimo una seguenza in modo tale che la seconda catena prenda come input l'output della prima

overall_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose = True)
# Specifichiamo output solo della prima chain 
output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Sure, here is an example of a function in Python that implements linear regression:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    numerator = 0
    denominator = 0

    for i in range(n):
        numerator += (x[i] - x_mean) * (y[i] - y_mean)
        denominator += (x[i] - x_mean) ** 2

    slope = numerator / denominator
    intercept = y_mean - slope * x_mean

    return slope, intercept

# Example usage
x = [1, 2, 3, 4, 5]
y = [2, 3, 4, 5, 6]

slope, intercept = linear_regression(x, y)
print("Slope:", slope)
print("Intercept:", intercept)
```

This function calculates the slope and intercept of the line that best fits the given data points `x` and `y` using the method of least squares. The resulting line represents the linear relationship between the two variables.
The provided Python code defines a function called `linear_regression`, which takes two arg

In [39]:
print(output['output'])

The provided Python code defines a function called `linear_regression`, which takes two arguments: lists (or arrays) `x` and `y`. These arguments correspond to the independent (predictor) and dependent (outcome) variables of a dataset, respectively. The function implements the method of least squares to compute the best-fitting line through the dataset, specifically finding the slope and intercept of that line. Let's break down the function's workings in detail:

1. **Import Necessary Library**:
   - The code begins by importing `numpy` as `np`. NumPy is a fundamental package for scientific computing in Python, offering support for a wide range of mathematical operations on arrays.

2. **Define the `linear_regression` Function**:
   - This function takes two parameters, `x` and `y`, which represent the independent and dependent variables.

3. **Variable Initialization and Preprocessing**:
   - `n`: The total number of observations/data points in the datasets `x` and `y`. It's calculate

## LangChain Agents

Gli LLM sono molto potenti, ma mancano di alcune capacità di performare semplici compiti, ad esempio faticano con la logica, e con il comunicare con componenti esterni. Ad esempio calcolare un elevamento a potenza lo potrebbe confondere.
Chiedo a gpt:
La risposta a 5.1^7.3 (5.1 elevato alla 7.3) è circa:
5.1^{7.3} \approx 781781.92

Ma se lo eseguo in python o qualsiasi calcolatrice: 146306,05007233

Questo avviene perché GPT usa l'approssimazione invece python usa la formula esatta

### LangChain Agents in Action: Python REPL

In [ ]:
pip install -q langchain_experimental

In [40]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_repl.run('print([n for n in range(1,100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [41]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

agent_executor = create_python_agent(
    llm= llm,
    tool = PythonAstREPLTool(), # sono funzioni che gli agenti usano per interagire con tool esterni
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the math module's factorial function. Then, I will calculate the square root of that result using the math.sqrt function. Finally, I will format the result to display it with 4 decimal points using the format function.
Action: python_repl_ast
Action Input: import math; result = math.sqrt(math.factorial(12)); format(result, '.4f')
Observation: 21886.1052
I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')

In [ ]:
# La risposta contiene un dizionario python che contiene input e output
print(response['input']), print(response['output'])

## LangChain Tools: DuckDuckGo and Wikipedia
I `Langchain tools` sono come app specializzate per il nostro LLM. Sono piccoli moduli di codice che ci permettono di accedere a informazioni o servizi.

Questi tools connettono il nostro LLM ai motori di ricerca, database, apis, e altro, espandendo le sue conoscenze e capacità.

### DuckDuckGo

In [ ]:
pip install -q duckduckgo-search

In [ ]:
pip install --upgrade duckduckgo_search

In [42]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

output = search.invoke('Where was Freddie Mercury born?')
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock 's most dynamic front men. Who is Freddie Mercury? Freddie Mercury, born Farrokh Bulsara on September 5, 1946, in Zanzibar, was a legendary singer-songwriter and the charismatic frontman of the rock band Queen. With a unique background as a Parsi in Tanzania and a music education in India, Mercury made a significant impact on the music world. Known for his extraordinary four-octave vocal range and theatrical stage ... Freddie Mercury was undoubtedly one of - if not the - greatest frontman of all time. Freddie Mercury helped make Queen one of the biggest ever rock bands, and he is regarded as one of the greatest singers in the history of rock music. Freddie Mercury, born Farrokh Bulsara on September 5, 19

In [43]:
search.name

'duckduckgo_search'

In [44]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

`DuckDuckGoSearchResult` per avere più informazioni e ricerche di link, avremo un oggetto composto da snippet + link

In [45]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()

output = search.run('Freddie Mercury and Queen')
print(output)

snippet: Freddie Mercury was the charismatic front man of Queen who captivated audiences with his extraordinary vocal range and showmanship, leaving an enduring legacy in rock music., title: Freddie Mercury | Biography, Parents, Songs, & Facts | Britannica, link: https://www.britannica.com/biography/Freddie-Mercury, snippet: Queen Roger Taylor Freddie Mercury Brian May John Deacon This is a list of all songs recorded by Queen., title: List of songs recorded by Queen - Wikipedia, link: https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Queen, snippet: Queen's iconic performance at Live Aid in 1985 was completely improvised by Freddie Mercury, and it left Brian May absolutely stunned., title: "We Didn't Plan It": Brian May Recalls the Iconic Freddie Mercury ..., link: https://americansongwriter.com/we-didnt-plan-it-brian-may-recalls-the-iconic-freddie-mercury-moment-that-was-completely-improvised/, snippet: Queen guitarist Brian May shares his memories of his late friend and bandmat

Per avere più controllo sul risultato usiamo un `wrapper`, in cui specifichiamo vari parametri.

In [46]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='it-it', max_results = 3, safesearch='moderate')

search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')

output = search.run('Palermo')

In [47]:
print(output)

snippet: Palermo is Sicily's cultural, economic and tourism capital. It is a city rich in history, culture, art, music and food., title: Palermo - Wikipedia, link: https://en.wikipedia.org/wiki/Palermo, snippet: Edizione di Palermo del Giornale di Sicilia, con notizie in tempo reale su cronaca, politica, calcio e sport con foto, video, approfondimenti e inchieste., title: Cronaca di Palermo, notizie in tempo reale - Giornale di Sicilia, link: https://palermo.gds.it/, snippet: Scopri cosa vedere a Palermo in questo itinerario di 1 giorno a piedi. Salva la mappa e prolunga il tuo viaggio con le gite nei dintorni., title: Cosa Vedere a Palermo in 1 Giorno: itinerario (con Mappa), link: https://www.eleonoraongaro.it/cosa-vedere-a-palermo-in-1-giorno-itinerario/, snippet: Scoprite cosa vedere e fare a Palermo e dintorni in 5 o più giorni di visita: i luoghi imperdibili da visitare fra la Palermo Araba Normanna e le altre attrazioni - Itinerario con info e mappe, dove mangiare., title: Cosa 

Possiamo anche formattare la risposta, usando espressioni regolari per avere una risposta più leggibile

In [48]:
import re
pattern = r"snippet: (.*?), title: (.*?), link: (https?://\S+)"

matches = re.findall(pattern, output, re.DOTALL) # re.DOTALL viene usato per matchare ogni linea

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Palermo is Sicily's cultural, economic and tourism capital. It is a city rich in history, culture, art, music and food.
Title: Palermo - Wikipedia
Link: https://en.wikipedia.org/wiki/Palermo,

--------------------------------------------------
Snippet: Edizione di Palermo del Giornale di Sicilia, con notizie in tempo reale su cronaca, politica, calcio e sport con foto, video, approfondimenti e inchieste.
Title: Cronaca di Palermo, notizie in tempo reale - Giornale di Sicilia
Link: https://palermo.gds.it/,

--------------------------------------------------
Snippet: Scopri cosa vedere a Palermo in questo itinerario di 1 giorno a piedi. Salva la mappa e prolunga il tuo viaggio con le gite nei dintorni.
Title: Cosa Vedere a Palermo in 1 Giorno: itinerario (con Mappa)
Link: https://www.eleonoraongaro.it/cosa-vedere-a-palermo-in-1-giorno-itinerario/,

--------------------------------------------------
Snippet: Scoprite cosa vedere e fare a Palermo e dintorni in 5 o più giorni di vi

### Wikipedia

In [ ]:
pip install -q wikipedia

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Prompt engineering\nSummary: Prompt engineering is the process of structuring or crafting an instruction in order to produce the best possible output from a generative artificial intelligence (AI) model.\nA prompt is natural language text describing the task that an AI should perform. A prompt for a text-to-text language model can be a query, a command, or a longer statement including context, instructions, and conversation history. Prompt engineering may involve phrasing a query, specifying'

In [5]:
wiki.invoke('Google Gemini')

"Page: Gemini (chatbot)\nSummary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name, it was launched in 2023 in response to the rise of OpenAI's ChatGPT. It was previously based on the LaMDA and PaLM LLMs.\nLaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popular"

## Creating a ReAct Agent
**ReAct** è un nuovo approccio che combina il *reasoning* (chain-of-thoughts prompting) e le capacità di azione degli LLMs

Con ReAct LLMs vengono generate tracce di ragionamento e azioni specifiche per attività in modo interlacciato.

**LangChain Agent: Tools + Chains**

In base all'input sarà in gradi di decidere quale strumento recuta più adatto, ad esempio DuckDuckGo o Wikipedia

**langchainhub** ci da accesso a varie risorse condivise. prompt, chains e agenti pre allenati, che possiamo usare.

In [6]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [61]:
pip install -U langchain langchain-openai

  Using cached langchain-0.3.22-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached openai-1.70.0-py3-none-any.whl.metadata (25 kB)
Using cached langchain-0.3.22-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.49-py3-none-any.whl (420 kB)
Using cached langchain_text_splitters-0.3.7-py3-none-any.whl (32 kB)
Using cached openai-1.70.0-py3-none-any.whl (599 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.63.0
    Uninstalling openai-1.63.0:
      Successfully uninstalled openai-1.63.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langch

In [8]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)


True

In [9]:
langsmith_key = os.getenv('LANGSMITH_API_KEY')

In [10]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
import os

# Disabilita tutte le opzioni di tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'false'
os.environ['LANGSMITH_TRACING'] = 'false'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = langsmith_key

os.environ['LANGSMITH_PROJECT'] = "corso-lang"
# llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = '''
Answer the following qustions as best as you can.
Questions: {q}
'''


prompt_template = PromptTemplate.from_template(template)

prompt_template = PromptTemplate.from_template(template)
# usiamo la struttura presa da una repo
prompt = hub.pull('hwchase17/react')
print(type(prompt))
print(prompt.input_variables) # agent_scratchpad, input, tool_name, tools

print(template) # ci mostra la catena di pensieri

## Useremo 3 tools
# 1. Python REPL tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func = python_repl.run,
    description = 'Useful when you need to use Python to answer a question. You should input Python code.'
)
# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

wikipedia_tool = Tool(
    name = 'Wikipedia',
    func = wikipedia.run,
    description = 'Useful when you need to look up a topic, country, or peson on Wikipedia'
)

# 3. DuckDuckGo Search Tool (for general web searches)

search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description = 'Useful for when you need to perform an internet search to find information that another tool can\'t provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm,tools,prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']

Answer the following qustions as best as you can.
Questions: {q}



In [6]:
question = "Generate the first 20 numbers in the Fibonacci series"
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I should use Python to generate the Fibonacci series.
Action: Python REPL
Action Input: 

```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        fib_series.append(fib_series[i-1] + fib_series[i-2])
    return fib_series

fibonacci(20)
```
I now know the final answer
Final Answer: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

> Finished chain.


In [13]:
question2 = "Tell me about Napoleon Bonaparte early life"
output2 = agent_executor.invoke({
    'input': prompt_template.format(q=question2)
})



> Entering new AgentExecutor chain...
I should use Wikipedia to look up information about Napoleon Bonaparte's early life.
Action: Wikipedia
Page: Napoleon IIIte early life"
Summary: Napoleon III (Charles-Louis Napoléon Bonaparte; 20 April 1808 – 9 January 1873) was President of France from 1848 to 1852 and then Emperor of the French from 1852 until his deposition in 1870. He was the first president, second emperor, and last monarch of France.
Prior to his reign, Napoleon III was known as Louis Napoleon Bonaparte. He was born in the Tuileries Palace at Paris as the son of Louis Bonaparte, King of Holland (r. 1806–1810), and Hortense de Beauharnais. Napoleon I was Louis Napoleon's paternal uncle, and one of his cousins was the disputed Napoleon II. It would only be two months following his birth that he, in accordance to Napoleon I's dynastic naming policy, would be bestowed the name of Charles-Louis Napoleon, however, shortly after, Charles was very quick to be removed from his name.

In [11]:
question = "Who is the current prime minister of the UK"
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
I should use DuckDuckGo Search to find the answer to this question.
Action: DuckDuckGo Search
What to know about Keir Starmer, UK's new prime minister by Yash Roy - 07/06/24 10:42 AM ET. by Yash Roy - 07/06/24 10:42 AM ET. ... During his first speech as prime minister, he promised that ... Mr Starmer has been leader of the Labour party since 2020 and is the country's 58th prime minister. He was born in 1962. His dad was a toolmaker in a factory and his mum was a nurse in the NHS. Sir Keir Starmer became Prime Minister on 5 July 2024. Education Keir attended Reigate Grammar School, before studying Law at the University of Leeds. He went on to do postgraduate studies at the ... The soon-to-be new UK Prime Minister Keir Starmer has addressed cheering supporters at a Labour Party victory rally in central London, and pledged "national renewal" following 14 years of ... Prime Minister Sir Keir Starmer — the former lawyer was knighted for services to cr

In [14]:
print(output['input'])


Answer the following qustions as best as you can.
Questions: Who is the current prime minister of the UK



In [15]:
print(output['output'])

Keir Starmer
